In [250]:
from io import StringIO

from geopy.geocoders import Nominatim
import pandas as pd
import requests
import seaborn as sns

sns.set(style="whitegrid")

%matplotlib inline

## Load data
We need to load COVID-19 data from an online source.

In [296]:
url = "https://datahub.io/core/covid-19/r/time-series-19-covid-combined.csv"
# Datahub blocks requests coming from Python, so we specify a different user-agent here
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
req = requests.get(url, headers=headers)
data = StringIO(req.text)

covid_data = pd.read_csv(
    data,
    parse_dates=True
)

In [198]:
covid_data.shape

(18576, 8)

In [199]:
covid_data.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0


In [297]:
# Make sure our Date column is treated as a proper DateTime
covid_data["Date"] = pd.to_datetime(covid_data["Date"])

In [201]:
covid_data.dtypes

Date              datetime64[ns]
Country/Region            object
Province/State            object
Lat                      float64
Long                     float64
Confirmed                float64
Recovered                float64
Deaths                   float64
dtype: object

In [202]:
covid_data["Date"]

0       2020-01-22
1       2020-01-23
2       2020-01-24
3       2020-01-25
4       2020-01-26
           ...    
18571   2020-03-29
18572   2020-03-30
18573   2020-03-31
18574   2020-04-01
18575   2020-04-02
Name: Date, Length: 18576, dtype: datetime64[ns]

## Look at most recent reports
For this initial experiment, we will get the most recent reports.

In [298]:
most_recent_reports = covid_data[covid_data["Date"] == covid_data.Date.max()]

In [300]:
# make sure we have only one unique date
most_recent_reports["Date"].value_counts()

2020-04-02    258
Name: Date, dtype: int64

In [301]:
most_recent_reports

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths
71,2020-04-02,Afghanistan,NaN,33.0000,65.0000,273.0,10.0,6.0
143,2020-04-02,Albania,NaN,41.1533,20.1683,277.0,76.0,16.0
215,2020-04-02,Algeria,NaN,28.0339,1.6596,986.0,61.0,86.0
287,2020-04-02,Andorra,NaN,42.5063,1.5218,428.0,10.0,15.0
359,2020-04-02,Angola,NaN,-11.2027,17.8739,8.0,1.0,2.0
...,...,...,...,...,...,...,...,...
18287,2020-04-02,Venezuela,NaN,6.4238,-66.5897,146.0,43.0,5.0
18359,2020-04-02,Vietnam,NaN,16.0000,108.0000,233.0,75.0,0.0
18431,2020-04-02,West Bank and Gaza,NaN,31.9522,35.2332,161.0,18.0,1.0
18503,2020-04-02,Zambia,NaN,-15.4167,28.2833,39.0,0.0,1.0


## Create summary statistics
Some countries are in the data multiple times. Here, we will sum up all the cases grouped by Country/Region.

In [223]:
aggregation = {
    "Confirmed": sum,
    "Recovered": sum,
    "Deaths": sum,
}

recent_country_level_statistics = most_recent_confirmed_cases.groupby("Country/Region").agg(aggregation)

In [224]:
recent_country_level_statistics

,Confirmed,Recovered,Deaths
Country/Region,,,
Afghanistan,273.0,10.0,6.0
Albania,277.0,76.0,16.0
Algeria,986.0,61.0,86.0
Andorra,428.0,10.0,15.0
Angola,8.0,1.0,2.0
...,...,...,...
Venezuela,146.0,43.0,5.0
Vietnam,233.0,75.0,0.0
West Bank and Gaza,161.0,18.0,1.0


## Create proportional statistics
Our mapping library wants all of the heatmap values to be scaled between 0 and 1. Here, we calculate the proportional values for each of the reported case types.

In [226]:
max_confirmed = recent_country_level_statistics["Confirmed"].max()
max_recovered = recent_country_level_statistics["Recovered"].max()
max_deaths = recent_country_level_statistics["Deaths"].max()

In [245]:
def get_proportional_case_confirmed(row):
    confirmed = row["Confirmed"]
    recovered = row["Recovered"]
    deaths = row["Deaths"]
    
    proportion_confirmed = confirmed / max_confirmed
    proportion_recovered = recovered / max_recovered
    proportion_deaths = deaths / max_deaths
    
    return pd.Series([proportion_confirmed, proportion_recovered, proportion_deaths])

In [246]:
new_columns = [
    "proportional_confirmed",
    "proportional_recovered",
    "proportional_deaths",
]
recent_country_level_statistics[new_columns] = recent_country_level_statistics.apply(get_proportional_case_confirmed, axis=1)

In [247]:
recent_country_level_statistics

,Confirmed,Recovered,Deaths,proportional_case_confirmed,proportional_confirmed,proportional_recovered,proportional_deaths
Country/Region,,,,,,,
Afghanistan,273.0,10.0,6.0,0.001121,0.001121,0.000131,0.000431
Albania,277.0,76.0,16.0,0.001138,0.001138,0.000993,0.001150
Algeria,986.0,61.0,86.0,0.004050,0.004050,0.000797,0.006180
Andorra,428.0,10.0,15.0,0.001758,0.001758,0.000131,0.001078
Angola,8.0,1.0,2.0,0.000033,0.000033,0.000013,0.000144
...,...,...,...,...,...,...,...
Venezuela,146.0,43.0,5.0,0.000600,0.000600,0.000562,0.000359
Vietnam,233.0,75.0,0.0,0.000957,0.000957,0.000980,0.000000
West Bank and Gaza,161.0,18.0,1.0,0.000661,0.000661,0.000235,0.000072


In [302]:
# Take a quick look at countries with highest confirmed cases
recent_country_level_statistics[recent_country_level_statistics["proportional_confirmed"] > 0.3]

,Confirmed,Recovered,Deaths,proportional_case_confirmed,proportional_confirmed,proportional_recovered,proportional_deaths,latitude,longitude
Country/Region,,,,,,,,,
China,82432.0,76565.0,3322.0,0.338595,0.338595,1.000000,0.238735,35.000074,104.999927
Germany,84794.0,22440.0,1107.0,0.348297,0.348297,0.293084,0.079554,51.083420,10.423447
Italy,115242.0,18278.0,13915.0,0.473364,0.473364,0.238725,1.000000,42.638426,12.674297
Spain,112065.0,26743.0,10348.0,0.460315,0.460315,0.349285,0.743658,39.326234,-4.838065
US,243453.0,9001.0,5926.0,1.000000,1.000000,0.117560,0.425871,39.783730,-100.445882


## Geocode countries
When we aggregated by Country/Region, we lost the longitude and latitude data. Here, we will use the OpenStreetMap Nominatum to get lat/lon data for each country.

In [295]:
def geocode_country(row):
    geocoder = Nominatim(user_agent="corvid_mapper")
    
    country = row.name
    
    result = geocoder.geocode(country)
    
    return pd.Series([result.latitude, result.longitude])

In [293]:
new_columns = [
    "latitude",
    "longitude"
]

recent_country_level_statistics[new_columns] = recent_country_level_statistics.apply(geocode_country, axis=1)

In [294]:
recent_country_level_statistics

,Confirmed,Recovered,Deaths,proportional_case_confirmed,proportional_confirmed,proportional_recovered,proportional_deaths,latitude,longitude
Country/Region,,,,,,,,,
Afghanistan,273.0,10.0,6.0,0.001121,0.001121,0.000131,0.000431,33.768006,66.238514
Albania,277.0,76.0,16.0,0.001138,0.001138,0.000993,0.001150,41.000028,19.999962
Algeria,986.0,61.0,86.0,0.004050,0.004050,0.000797,0.006180,28.000027,2.999983
Andorra,428.0,10.0,15.0,0.001758,0.001758,0.000131,0.001078,42.540717,1.573203
Angola,8.0,1.0,2.0,0.000033,0.000033,0.000013,0.000144,-11.877577,17.569124
...,...,...,...,...,...,...,...,...,...
Venezuela,146.0,43.0,5.0,0.000600,0.000600,0.000562,0.000359,8.001871,-66.110932
Vietnam,233.0,75.0,0.0,0.000957,0.000957,0.000980,0.000000,13.290403,108.426511
West Bank and Gaza,161.0,18.0,1.0,0.000661,0.000661,0.000235,0.000072,31.433166,34.377929
